# Investigating the relationship between food deserts and COVID19 deaths in Denver

## Introduction

## Data sources

Download and import all dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install bs4 # Comment out when installed
from bs4 import BeautifulSoup # library to scrape website

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from sklearn for clustering analysis
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 122kB 16.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              cond

### Get data on Denver neighborhoods

In [23]:
# Download statistical neighborhood shapefile
import wget
url = 'https://www.denvergov.org/media/gis/DataCatalog/statistical_neighborhoods/shape/statistical_neighborhoods.zip'
wget.download(url)

# Unzip folder to get files
from zipfile import ZipFile
with ZipFile('statistical_neighborhoods.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

# Convert shapefile to geojson for folium usage
!pip install pyshp
import shapefile
Denver_statneigh_geojson = shapefile.Reader("statistical_neighborhoods.zip").__geo_interface__

In [ ]:
# !pip install shapely
import os
import json
from shapely.geometry import shape, Point, Polygon

def read_geojson(filepath):
    with open(filepath, 'r') as f:
        js = json.load(f)
    return js

# get boundary from a geojson file
# output format: (minx, miny, maxx, maxy) tuple (float values) 
def get_boundary_from_geojson(geojson):
    js = read_geojson(geojson) if isinstance(geojson, str) else geojson
    polygon = [shape(feature['geometry']).bounds for feature in js['features']]

    boundary = []
    for p in polygon:
        boundary.append((p[0], p[1]))
        boundary.append((p[2], p[3]))
    return Polygon(boundary).bounds

# Given a point, check whether it's within the boundaries defined by GeoJSON file
# input: point = (longtitude, latitude)
# output: True if in polygon of geojson file
def is_in_geojson(geojson, point, verbose=False):    
    js = read_geojson(geojson) if isinstance(geojson, str) else geojson
    pos = point if isinstance(point, Point) else Point(point)

    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(pos):
            if verbose: print('Found in borough: ', feature['properties'])
            return True
    return False

# load GeoJSON file containing montreal boroughs
geoJson = load_dict(streaming_body_1) if From_IBM_Cloud else os.path.join(os.getcwd(), 'montreal_shapefile.geojson')
# get the boundaries of Montreal island
montreal_boundary = get_boundary_from_geojson(geoJson)
print(f'The boundaries of Montreal island: {montreal_boundary}\n')

# check whether montreal center is on Montreal island
point = Point((montreal_center[1], montreal_center[0]))
print(f'Montreal center ({point.x}, {point.y}) in geojson file: {is_in_geojson(geoJson, point, True)}')

### Make a map of Denver neighborhoods

In [2]:
# Use geopy to get a user_agent and data for Denver
address = 'Denver, Colorado'
geolocator = Nominatim(user_agent="Denver_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver, Colorado are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver, Colorado are 39.7392364, -104.9848623.


In [3]:
# create map of Denver using latitude and longitude values
map_Denver = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Denver

# add markers to map
for lat, lng, borough, neighborhood in zip(TorNeighdf_full['Latitude'], TorNeighdf_full['Longitude'], TorNeighdf_full['Borough'], TorNeighdf_full['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

Denver has "statistical neighborhood" boundaries that are available at https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-statistical-neighborhoods with various filetypes and metadata. We will download the shapefile and convert to GeoJSON to use with folium.

### Generate candidate neighborhoods for food desert analysis

We will use a popular hexagon honeycomb grid on map, each grid of cell represents a candidate of interest, the length of side is 10003√ meters, it covers around area of 0.87 square kilometers. Since we only focus on Montreal island, we will calculate the honeycomb grid covering the island only.

Firstly, we will find the latitude & longitude of Montreal city center using Google Geocoding API.

Next, Montreal sits on an island, we only focus on the area in the Montreal island, so try to find the boundary of Montreal island first.
We download Montreal shapefile (in geojson format) from Carto which defines boundaries of boroughs and municipalities on Montreal island.

When we generate candidate hexagon grid, we will limit the hexagons within island's boundary. So we will define several convenient methods first.

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
def load_dict(fileobject):
    '''Load the file contents into a Python dict'''
    text = fileobject.read()
    dictformat = json.loads(text)
    return dictformat

### Convert neighborhood shape file to GeoJSON and import into folium

In [ ]:
Looks good.

But we also can notice it uses latitude and longitude degrees(WGS84 spherical coordinate system) in the geojson definition file, not the common metric unit - meter or kilometer which is UTM Cartesian coordinate system (X/Y coordinates in meters).

So we will create several methods to convert latitude and longitude degrees into meters and vice versa.

In [ ]:
# !pip install shapely
import shapely.geometry

# !pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=18, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=18, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

def calc_lonlat_distance(lon1, lat1, lon2, lat2):
    x1, y1 = lonlat_to_xy(lon1, lat1)
    x2, y2 = lonlat_to_xy(lon2, lat2)
    return calc_xy_distance(x1, y1, x2, y2)

print('Coordinate transformation check')
print('-------------------------------')
print('Montreal center longitude = {}, latitude = {}'.format(montreal_center[1], montreal_center[0]))
x, y = lonlat_to_xy(montreal_center[1], montreal_center[0])
print('Montreal center UTM X = {}, Y = {}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Montreal center longitude = {}, latitude = {}'.format(lo, la))
distance = calc_lonlat_distance(*montreal_boundary)
print(f'Distance from most southwest point to most northeast point in Montreal is {distance} meters.')
mtl_min_x, mtl_min_y = lonlat_to_xy(montreal_boundary[0], montreal_boundary[1])
mtl_max_x, mtl_max_y = lonlat_to_xy(montreal_boundary[2], montreal_boundary[3])
print(f'Distance from west to east in Montreal is {mtl_max_x-mtl_min_x} meters.')
print(f'Distance from north to south in Montreal is {mtl_max_y-mtl_min_y} meters.')

Now we can create a honeycomb grid of cells with regular hexagons. The centers of hexagon are equally distant in one row, and we offset every other row. So that we covers all the areas on the island without spacing among candidate hexagons. And every cell center is same distant from all its neighbors' center.

In this project, we define length of side of the hexagon with 1000/√3 meters. From geometric, we can calculate the distance between the centers of two adjacent hexagons in one row which is 1000 meters

In [ ]:
mtl_center_x, mtl_center_y = lonlat_to_xy(montreal_center[1], montreal_center[0]) # City center in Cartesian coordinates
montreal_boroughs = geoJson if From_IBM_Cloud else read_geojson(geoJson)

k = math.sqrt(3) / 2 # Vertical distance coefficient for hexagon grid cells
x_step = 1000        # Distance between adjacent hexagons in the row
y_step = x_step * k  # Distance between adjacent hexagons in vertical direction

latitudes = []
longitudes = []
distances_from_downtown = []
xs = []
ys = []
num_columns = int((mtl_max_x - mtl_min_x)/x_step) + 1
num_rows = int((mtl_max_y - mtl_min_y)/y_step) + 1
for i in range(0, num_rows):
    y = mtl_min_y + i * y_step
    x_offset = x_step/2 if i%2==0 else 0
    for j in range(0, num_columns):
        x = mtl_min_x + j * x_step + x_offset
        lon, lat = xy_to_lonlat(x, y)
        if is_in_geojson(montreal_boroughs, (lon, lat)):
            distance = calc_xy_distance(mtl_center_x, mtl_center_y, x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_downtown.append(distance)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers are generated.')

Calculate the boundary of hexagons:

In [ ]:
import numpy as np
import math

def get_hexagon_coordinates(centroid_latlon, side_length_in_meters, rotate=False):
    # Rotate 90 degree
    unit_hexagon_vertices_rotate = np.array([[0, 1], [-math.sqrt(3)/2, 1/2], [-math.sqrt(3)/2, -1/2],
                                       [0, -1], [math.sqrt(3)/2, -1/2], [math.sqrt(3)/2, 1/2]])
    # Regular hexagon
    unit_hexagon_vertices = np.array([[1, 0], [1/2, math.sqrt(3)/2], [-1/2, math.sqrt(3)/2],
                                       [-1, 0], [-1/2, -math.sqrt(3)/2], [1/2, -math.sqrt(3)/2]])
    x, y = lonlat_to_xy(centroid_latlon[1], centroid_latlon[0])
    vertices = unit_hexagon_vertices_rotate if rotate else unit_hexagon_vertices
    vertices *= side_length_in_meters
    vertices = [xy_to_lonlat(v[0]+x, v[1]+y) for v in vertices]
    return vertices     # list of 6 (longitude, latitude) tuples

a = x_step/math.sqrt(3)
hexagons = []
for latlon in zip(latitudes, longitudes):
    vertices = get_hexagon_coordinates(latlon, a, True)
    hexagons.append(Polygon(vertices))

print(len(hexagons), 'candidate hexagon neighborhoods generated.')

Visualize the data we have so far:

In [ ]:
# !pip install folium
import folium
from folium.features import DivIcon

def hexagon_style(feature):
    return { 'color': 'blue', 'fill': False }
def boroughs_style(feature):
    return { 'color': 'gray' }

center = xy_to_lonlat((mtl_max_x+mtl_min_x)/2, (mtl_max_y+mtl_min_y)/2)
map_montreal = folium.Map(location=(center[1], center[0]), zoom_start=11)
folium.Marker(montreal_center, popup='Downtown Montreal').add_to(map_montreal)
folium.GeoJson(montreal_boroughs, style_function=boroughs_style, name='geojson').add_to(map_montreal)
for i, hexagon in enumerate(hexagons):
    folium.GeoJson(hexagon.convex_hull, style_function=hexagon_style).add_to(map_montreal)
    # folium.map.Marker((hexagon.centroid.y, hexagon.centroid.x),
    #                   icon=DivIcon(html=f'<div style="font-size: 12pt">{i}</div>', )
    #                  ).add_to(map_montreal)
map_montreal

In [2]:
# Use the requests library to download the List of postal codes of Canada: M webpage
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

# Parse data using beautifulsoup
TorontoNeigh_soup = BeautifulSoup(html_data, "html5lib")

Extract all of the tables from the webpage then assign/parse the relevant table

In [ ]:
# Find all of the tables using find_all(), determine number of tables and extract relevant index if necessary, print out table with .prettify() and check index of relevant columns for extraction
tables_all = TorontoNeigh_soup.find_all('table')
#len(tables_all) # Determine how many total tables there are
#print(tables_all[0]) # Change index and print to determine which table is the correct one

# Make table for Toronto neighborhoods and extract relevant table from TorontoNeigh_soup
TorontoNeigh_table=[]
TorPost_table=tables_all[0]

# Parse the table and assign to relevent columns
for row in TorPost_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        TorontoNeigh_table.append(cell)

# Convert to pandas dataframe and handle
TorNeigh_df=pd.DataFrame(TorontoNeigh_table)
TorNeigh_df['Borough']=TorNeigh_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
TorNeigh_df

Get latitude and longitude for the postal codes from csv file provided then merge with neighborhood data on postal code

In [4]:
with open('Geospatial_Coordinates.csv') as Geo_Coords:
    toronto_geodata = pd.read_csv(Geo_Coords)

toronto_geodata.rename(columns={'Postal Code':'PostalCode','Latitude':'Latitude','Longitude':'Longitude'}, inplace=True)
TorNeighdf_full = pd.merge(TorNeigh_df, toronto_geodata, how='inner', on=["PostalCode"])
print(TorNeighdf_full.head())
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorNeighdf_full['Borough'].unique()),
        TorNeighdf_full.shape[0]
    )
)

  PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
The dataframe has 15 boroughs and 103 neighborhoods.


### Set up Foursquare Credentials and get location data for the neighborhoods

In [ ]:
CLIENT_ID = 'YVBPCW4IZS3RXMMLWV03H0H5NHCWTID5DZLMONQ3WAZJ222R' # your Foursquare ID
CLIENT_SECRET = 'YYBZ02FFK24FZ1JYIJPCG4M54BL4FZCMLQECPN5RIE4RCPMP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
neighborhood_latitude = TorNeighdf_full.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorNeighdf_full.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorNeighdf_full.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

### Make function and extract venue information for neighborhoods

In [ ]:
# Set limit/radius for analysis
limit = 100
radius = 500

grocery_store='4bf58dd8d48988d118951735'
organic_grocery='52f2ab2ebcbc57f1066b8b45'
health_foodstore='50aa9e744b90af0d42d5de0e'
supermarket='52f2ab2ebcbc57f1066b8b46'
fruit_vegetablestore='52f2ab2ebcbc57f1066b8b1c'

# Create function for getting venue information for all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [ ]:
# Fetch venue information for each neighborhood
toronto_venues = getNearbyVenues(names=TorNeighdf_full['Neighborhood'], latitudes=TorNeighdf_full['Latitude'], longitudes=TorNeighdf_full['Longitude'], radius=radius)

In [ ]:
# Check totals, how many unique and how many of venues are in each neighborhood
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

In [ ]:
# Get one hot encoding for Toronto venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
# Create function to get the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# Find top 10 most common venues for each neighborhoods
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # for 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) # for the rest

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

## Methods

## Results

### Analyze Denver Neighborhoods

### Run elbow method to determine optimal number of clusters

In [ ]:
# Import yellowbrick wrapper for sklearn and ElbowVisualizer/Silhouette plotters
#!pip install -U yellowbrick
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
model = KMeans()

# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(1,12), timings=False, random_state=4)
visualizer.fit(toronto_grouped_clustering)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
# set number of clusters, graph isn't great for discernment but going ahead with k=6 anyway
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = TorNeighdf_full

# merge toronto_grouped with TorNeighdf_full to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# remove rows with NaN as cluster label and assign int to the cluster labels rather than float
toronto_merged.dropna(axis=0, how='any', subset=['Cluster Labels'], inplace=True)
toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [ ]:
### Examine Clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]